# Train Merlin Two-Towers

### pip & package

In [1]:
import os
# import nvtabular as nvt
from time import time
import pandas as pd
# disable INFO and DEBUG logging everywhere
import logging
import time
from pprint import pprint

logging.disable(logging.WARNING)

# from nvtabular.ops import (
#     Categorify,
#     TagAsUserID,
#     TagAsItemID,
#     TagAsItemFeatures,
#     TagAsUserFeatures,
#     AddMetadata,
#     ListSlice
# )
# import nvtabular.ops as ops

# from merlin.schema.tags import Tags

# import merlin.models.tf as mm
# from merlin.io.dataset import Dataset
# from merlin.io.dataset import Dataset as MerlinDataset
# from merlin.models.utils.example_utils import workflow_fit_transform
# import tensorflow as tf

from google.cloud import aiplatform as vertex_ai
# from google.cloud.aiplatform import hyperparameter_tuning as hpt

# for running this example on CPU, comment out the line below
# os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

### Setup

In [2]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
PROJECT_NUM = !gcloud projects list --filter="$PROJECT_ID" --format="value(PROJECT_NUMBER)"
PROJECT_NUM = PROJECT_NUM[0]
LOCATION = 'us-central1'

print(f"PROJECT_ID: {PROJECT_ID}")
print(f"PROJECT_NUM: {PROJECT_NUM}")
print(f"LOCATION: {LOCATION}")

PROJECT_ID: hybrid-vertex
PROJECT_NUM: 934903580331
LOCATION: us-central1


In [3]:
# TODO: Service Account address
VERTEX_SA = '934903580331-compute@developer.gserviceaccount.com' # Change to your service account with Vertex AI Admin permitions.

In [5]:
# Bucket definitions
BUCKET = 'jt-merlin-scaling' # 'spotify-merlin-v1'

VERSION = 'jtv16'
MODEL_NAME = '2tower'
FRAMEWORK = 'merlin-tf'
MODEL_DISPLAY_NAME = f'vertex-{FRAMEWORK}-{MODEL_NAME}-{VERSION}'
WORKSPACE = f'gs://{BUCKET}/{MODEL_DISPLAY_NAME}'

# # Docker definitions for training
# IMAGE_NAME = f'{FRAMEWORK}-{MODEL_NAME}-training-{VERSION}'
# IMAGE_URI = f'gcr.io/{PROJECT_ID}/{IMAGE_NAME}'
# # DOCKERNAME = 'hugectr'
# DOCKERNAME = 'merlintf'
# MACHINE_TYPE ='e2-highcpu-32'
# FILE_LOCATION = './src'

# Training Package

In [6]:
REPO_DOCKER_PATH_PREFIX = 'src'
TRAIN_SUB_DIR = 'trainer'

In [7]:
# Make the training subfolder
! rm -rf {REPO_DOCKER_PATH_PREFIX}/{TRAIN_SUB_DIR}
! mkdir {REPO_DOCKER_PATH_PREFIX}/{TRAIN_SUB_DIR}

In [8]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/{TRAIN_SUB_DIR}/__init__.py
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Writing src/trainer/__init__.py


## Interactive Train Shell

In [9]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/{TRAIN_SUB_DIR}/interactive_train.py

import time

while(True):
    time.sleep(60)

Writing src/trainer/interactive_train.py


## Two-Tower Model

In [10]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/{TRAIN_SUB_DIR}/two_tower_model.py

from typing import List, Any

import nvtabular as nvt
# # import nvtabular.ops as ops

# from merlin.models.utils.example_utils import workflow_fit_transform
from merlin.schema.tags import Tags
import merlin.models.tf as mm
from merlin.models.tf.outputs.base import DotProduct, MetricsFn, ModelOutput

import logging

import tensorflow as tf


def create_two_tower(
    train_dir: str,
    valid_dir: str,
    workflow_dir: str,
    layer_sizes: List[Any] = [512, 256, 128],
):
    
    #=========================================
    # get workflow details
    #=========================================
    workflow = nvt.Workflow.load(workflow_dir) # gs://spotify-merlin-v1/nvt-preprocessing-spotify-v24/nvt-analyzed
    
    schema = workflow.output_schema
    # embeddings = ops.get_embedding_sizes(workflow)
    
    user_schema = schema.select_by_tag(Tags.USER)
    user_inputs = mm.InputBlockV2(user_schema)
    
    #=========================================
    # build towers
    #=========================================
    query = mm.Encoder(user_inputs, mm.MLPBlock(layer_sizes))
    
    item_schema = schema.select_by_tag(Tags.ITEM)
    item_inputs = mm.InputBlockV2(
        item_schema,
    )
    candidate = mm.Encoder(item_inputs, mm.MLPBlock(layer_sizes))
    
    model = mm.TwoTowerModelV2(
        query_tower=query,
        candidate_tower=candidate,
        # output=mm.ContrastiveOutput(
        #     to_call=DotProduct(),
        #     negative_samplers="in-batch",
        #     schema=item_schema.select_by_tag(Tags.ITEM_ID),
        #     candidate_name="item",
        # )
    )
    
    return model

Writing src/trainer/two_tower_model.py


## Train task

In [11]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/{TRAIN_SUB_DIR}/train_task.py

import argparse
import json
import logging
import os
import sys
import time
import pandas as pd

# we can control how much memory to give tensorflow with this environment variable
# IMPORTANT: make sure you do this before you initialize TF's runtime, otherwise
# TF will have claimed all free GPU memory
# os.environ["TF_MEMORY_ALLOCATION"] = "0.3"  # fraction of free memory

# # nvtabular
# import nvtabular as nvt
# import nvtabular.ops as ops

# merlin
# from merlin.models.utils.example_utils import workflow_fit_transform
from merlin.io.dataset import Dataset as MerlinDataset
from merlin.models.tf.outputs.base import DotProduct, MetricsFn, ModelOutput
from merlin.schema.tags import Tags
import merlin.models.tf as mm

from merlin.models.utils.dataset import unique_rows_by_features

# nvtabular
import nvtabular as nvt
import nvtabular.ops as ops

# tensorflow
import tensorflow as tf
from tensorflow.python.client import device_lib

# gcp
import google.cloud.aiplatform as vertex_ai
from google.cloud import storage
from google.cloud.storage.bucket import Bucket
from google.cloud.storage.blob import Blob
# import hypertune
# from google.cloud.aiplatform.training_utils import cloud_profiler

# repo
from .two_tower_model import create_two_tower
# import utils

# local
HYPERTUNE_METRIC_NAME = 'AUC'
LOCAL_MODEL_DIR = '/tmp/saved_model'
LOCAL_CHECKPOINT_DIR = '/tmp/checkpoints'

# ====================================================
# Helper functions - TODO: move to utils?
# ====================================================

def _is_chief(task_type, task_id): 
    ''' Check for primary if multiworker training
    '''
    if task_type == 'chief':
        results = 'chief'
    else:
        results = None
    return results
    # return (task_type == 'chief') or (task_type == 'worker' and task_id == 0) or task_type is None
    # return ((task_type == 'chief' and task_id == 0) or task_type is None)

def get_upload_logs_to_manged_tb_command(tb_resource_name, logs_dir, experiment_name, ttl_hrs, oneshot="false"):
    """
    Run this and copy/paste the command into terminal to have 
    upload the tensorboard logs from this machine to the managed tb instance
    Note that the log dir is at the granularity of the run to help select the proper
    timestamped run in Tensorboard
    You can also run this in one-shot mode after training is done 
    to upload all tb objects at once
    """
    return(
        f"""tb-gcp-uploader --tensorboard_resource_name={tb_resource_name} \
        --logdir={logs_dir} \
        --experiment_name={experiment_name} \
        --one_shot={oneshot} \
        --event_file_inactive_secs={60*60*ttl_hrs}"""
    )

def _upload_blob_gcs(gcs_uri, source_file_name, destination_blob_name, project):
    """Uploads a file to GCS bucket"""
    client = storage.Client(project=project)
    blob = Blob.from_string(os.path.join(gcs_uri, destination_blob_name))
    blob.bucket._client = client
    blob.upload_from_filename(source_file_name)
    
def get_arch_from_string(arch_string):
    q = arch_string.replace(']', '')
    q = q.replace('[', '')
    q = q.replace(" ", "")
    return [int(x) for x in q.split(',')]

# ====================================================
# TRAINING SCRIPT
# ====================================================
    
def main(args):
    """Runs a training loop."""
    
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
    # tf.debugging.set_log_device_placement(True) # logs all tf ops and their device placement;
    # os.environ['TF_GPU_THREAD_MODE']='gpu_private'
    # os.environ['TF_GPU_THREAD_COUNT']='1'
    os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
    
    TIMESTAMP = time.strftime("%Y%m%d-%H%M%S")
    
    vertex_ai.init(project=f'{args.project}', location=f'{args.location}')
    storage_client = storage.Client(project=args.project)
    logging.info("vertex_ai initialized...")
    
    EXPERIMENT_NAME = f"{args.experiment_name}"
    RUN_NAME = f"{args.experiment_run}" #-{TIMESTAMP}" # f"{args.experiment_run}"
    logging.info(f"EXPERIMENT_NAME: {EXPERIMENT_NAME}\n RUN_NAME: {RUN_NAME}")
    
    WORKING_DIR_GCS_URI = f'gs://{args.train_output_bucket}/{EXPERIMENT_NAME}/{RUN_NAME}'
    logging.info(f"WORKING_DIR_GCS_URI: {WORKING_DIR_GCS_URI}")
    
    TB_RESOURCE_NAME = f'{args.tb_name}'
    LOGS_DIR = f'{WORKING_DIR_GCS_URI}/tb_logs'
    logging.info(f"tensorboard LOGS_DIR: {LOGS_DIR}")
    
    # ====================================================
    # Set Device / GPU Strategy
    # ====================================================    
    logging.info("Detecting devices....")
    logging.info(f'Detected Devices {str(device_lib.list_local_devices())}')
    
    logging.info("Setting device strategy...")
    
    # Single Machine, single compute device
    if args.distribute == 'single':
        if tf.test.is_gpu_available(): # TODO: replace with - tf.config.list_physical_devices('GPU')
            strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
        else:
            strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
        logging.info("Single device training")
    
    # Single Machine, multiple compute device
    elif args.distribute == 'mirrored':
        strategy = tf.distribute.MirroredStrategy()
        logging.info("Mirrored Strategy distributed training")

    # Multi Machine, multiple compute device
    elif args.distribute == 'multiworker':
        strategy = tf.distribute.MultiWorkerMirroredStrategy()
        logging.info("Multi-worker Strategy distributed training")
        logging.info('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
        
    
    # set related vars...
    NUM_WORKERS = strategy.num_replicas_in_sync
    GLOBAL_BATCH_SIZE = NUM_WORKERS * args.per_gpu_batch_size
    # num_gpus = sum([len(gpus) for gpus in args.gpus])
    # GLOBAL_BATCH_SIZE = num_gpus * args.per_gpu_batch_size

    logging.info(f'NUM_WORKERS = {NUM_WORKERS}')
    # logging.info(f'num_gpus: {num_gpus}')
    logging.info(f'GLOBAL_BATCH_SIZE: {GLOBAL_BATCH_SIZE}')
    
    # set worker vars...
    logging.info(f'Setting task_type and task_id...')
    if args.distribute == 'multiworker':
        task_type, task_id = (
            strategy.cluster_resolver.task_type,
            strategy.cluster_resolver.task_id
        )
    else:
        task_type, task_id = 'chief', None
    
    logging.info(f'task_type = {task_type}')
    logging.info(f'task_id = {task_id}')
        
    # ====================================================
    # Prepare Train and Valid Data
    # ====================================================
    logging.info(f'Loading workflow & schema from : {args.workflow_dir}')
    
    workflow = nvt.Workflow.load(args.workflow_dir) # gs://{BUCKET}/..../nvt-analyzed
    schema = workflow.output_schema
    # embeddings = ops.get_embedding_sizes(workflow)
    
    train_data = MerlinDataset(os.path.join(args.train_dir, "*.parquet"), schema=schema, part_size="1GB")
    valid_data = MerlinDataset(os.path.join(args.valid_dir, "*.parquet"), schema=schema, part_size="1GB")
    
    # train_data = MerlinDataset(args.train_dir + "*.parquet", part_size="1GB")
    # valid_data = MerlinDataset(args.valid_dir + "*.parquet", part_size="1GB")
    
    # ====================================================
    # Callbacks
    # ====================================================
    class UploadTBLogsBatchEnd(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            os.system(
                get_upload_logs_to_manged_tb_command(
                    tb_resource_name=TB_RESOURCE_NAME, 
                    logs_dir=LOGS_DIR, 
                    experiment_name=EXPERIMENT_NAME,
                    ttl_hrs = 5, 
                    oneshot="true",
                )
            )
            
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=LOGS_DIR,
        histogram_freq=0, 
        write_graph=True, 
        # profile_batch=(20,50)
    )
    
    # ====================================================
    # Train
    # ==================================================== 
    LAYER_SIZES = get_arch_from_string(args.layer_sizes)
    logging.info(f'LAYER_SIZES: {LAYER_SIZES}')

    # with strategy.scope():
    model = create_two_tower(
        train_dir=args.train_dir,
        valid_dir=args.valid_dir,
        workflow_dir=args.workflow_dir,
        layer_sizes=LAYER_SIZES # args.layer_sizes,
    )
        
        
    model.compile(
        optimizer=tf.keras.optimizers.Adagrad(args.learning_rate),
        run_eagerly=False,
        metrics=[mm.RecallAt(1), mm.RecallAt(10), mm.NDCGAt(10)],
    )
    
    # cloud_profiler.init() # managed TB profiler
        
    logging.info('Starting training loop...')
    
    start_model_fit = time.time()
    
    model.fit(
        train_data, 
        validation_data=valid_data, 
        batch_size=GLOBAL_BATCH_SIZE, 
        epochs=args.num_epochs,
        # steps_per_epoch=20, 
        callbacks=[
            tensorboard_callback, 
            UploadTBLogsBatchEnd()
        ],
    )
    
    # capture elapsed time
    end_model_fit = time.time()
    elapsed_model_fit = end_model_fit - start_model_fit
    elapsed_model_fit = round(elapsed_model_fit, 2)
    logging.info(f'Elapsed model_fit: {elapsed_model_fit} seconds')
    
    # ====================================================
    # metaparams & metrics for Vertex Ai Experiments
    # ====================================================
    logging.info('Logging params & metrics for Vertex Experiments')
    
    # get the metrics for the experiment run
    history_keys = model.history.history.keys()
    metrics_dict = {}
    _ = [metrics_dict.update({key: model.history.history[key][-1]}) for key in history_keys]
    metrics_dict["elapsed_model_fit"] = elapsed_model_fit
    
    logging.info(f'metrics_dict: {metrics_dict}')
    
    metaparams = {}
    metaparams["experiment_name"] = f'{EXPERIMENT_NAME}'
    metaparams["experiment_run"] = f"{RUN_NAME}"
    
    logging.info(f'metaparams: {metaparams}')
    
    hyperparams = {}
    hyperparams["epochs"] = int(args.num_epochs)
    hyperparams["num_gpus"] = NUM_WORKERS # num_gpus
    hyperparams["per_gpu_batch_size"] = args.per_gpu_batch_size
    hyperparams["global_batch_size"] = GLOBAL_BATCH_SIZE
    hyperparams["learning_rate"] = args.learning_rate
    hyperparams['layers'] = f'{args.layer_sizes}'
    
    logging.info(f'hyperparams: {hyperparams}')
    
    # ====================================================
    # Experiments
    # ====================================================
    logging.info(f"Creating run: {RUN_NAME}; for experiment: {EXPERIMENT_NAME}")
    
    if task_type == 'chief':
        logging.info(f" task_type logging experiments: {task_type}")
        logging.info(f" task_id logging experiments: {task_id}")
    
        # Create experiment
        vertex_ai.init(experiment=EXPERIMENT_NAME)

        with vertex_ai.start_run(RUN_NAME) as my_run:
            logging.info(f"logging metrics_dict")
            my_run.log_metrics(metrics_dict)

            logging.info(f"logging metaparams")
            my_run.log_params(metaparams)

            logging.info(f"logging hyperparams")
            my_run.log_params(hyperparams)
        
    # =============================================
    # save retrieval (query) tower
    # =============================================
    # set vars...
    MODEL_DIR = f"{WORKING_DIR_GCS_URI}/model-dir"
    logging.info(f'Saving towers to {MODEL_DIR}')
    
    QUERY_TOWER_PATH = f"{MODEL_DIR}/query-tower"
    CANDIDATE_TOWER_PATH = f"{MODEL_DIR}/candidate-tower"
    EMBEDDINGS_PATH = f"{MODEL_DIR}/candidate-embeddings"
    
    if task_type == 'chief':
        # save query tower
        query_tower = model.query_encoder
        query_tower.save(QUERY_TOWER_PATH)
        logging.info(f'Saved query tower to {QUERY_TOWER_PATH}')
        
        candidate_tower = model.candidate_encoder
        candidate_tower.save(CANDIDATE_TOWER_PATH)
        logging.info(f'Saved candidate tower to {CANDIDATE_TOWER_PATH}')
    
    # =============================================
    # save embeddings for ME index
    # =============================================
    EMBEDDINGS_FILE_NAME = "candidate_embeddings.json"
    logging.info(f"Saving {EMBEDDINGS_FILE_NAME} to {EMBEDDINGS_PATH}")
    
    # def format_for_matching_engine(data) -> None:
    #     emb = [data[i] for i in range(LAYER_SIZES[-1])] # get the embeddings
    #     formatted_emb = '{"id":"' + str(data['track_uri_can']) + '","embedding":[' + ",".join(str(x) for x in list(emb)) + ']}'
    #     with open(f"{EMBEDDINGS_FILE_NAME}", 'a') as f:
    #         f.write(formatted_emb)
    #         f.write("\n")
    
    def format_for_matching_engine(data) -> None:
        cols = [str(i) for i in range(LAYER_SIZES[-1])]      # ensure we are only pulling 0-EMBEDDING_DIM cols
        emb = [data[col] for col in cols]                    # get the embeddings
        formatted_emb = '{"id":"' + str(data['track_uri_can']) + '","embedding":[' + ",".join(str(x) for x in list(emb)) + ']}'
        with open(f"{EMBEDDINGS_FILE_NAME}", 'a') as f:
            f.write(formatted_emb)
            f.write("\n")
    
    # !rm candidate_embeddings.json > /dev/null 
    # !touch candidate_embeddings.json
    item_data = pd.read_parquet(f'{args.workflow_dir}/categories/unique.track_uri_can.parquet')
    lookup_dict = dict(item_data['track_uri_can'])

    # item embeds from TRAIN
    start_embeds = time.time()
    
    item_features = (unique_rows_by_features(train_data, Tags.ITEM, Tags.ID))
    item_embs = model.candidate_embeddings(item_features, index=item_features.schema['track_uri_can'], batch_size=10000)
    item_emb_pd = item_embs.compute().to_pandas().fillna(1e-10).reset_index() #filling blanks with an epsilon value
    item_emb_pd['track_uri_can'] = item_emb_pd['track_uri_can'].apply(lambda l: lookup_dict[l])
    _ = item_emb_pd.apply(format_for_matching_engine, axis=1)
    
    # capture elapsed time
    end_embeds = time.time()
    elapsed_time = end_embeds - start_embeds
    elapsed_time = round(elapsed_time, 2)
    logging.info(f'Elapsed time writting TRAIN embeddings: {elapsed_time} seconds')
    
    # item embeds from VALID
    start_embeds = time.time()
    
    item_features_val = (unique_rows_by_features(valid_data, Tags.ITEM, Tags.ID))
    item_embs_val = model.candidate_embeddings(item_features_val, index=item_features_val.schema['track_uri_can'], batch_size=10000)
    item_emb_pd_val = item_embs_val.compute().to_pandas().fillna(1e-10).reset_index() #filling blanks with an epsilon value
    item_emb_pd_val['track_uri_can'] = item_emb_pd_val['track_uri_can'].apply(lambda l: lookup_dict[l])
    _ = item_emb_pd_val.apply(format_for_matching_engine, axis=1)
    
    # capture elapsed time
    end_embeds = time.time()
    elapsed_time = end_embeds - start_embeds
    elapsed_time = round(elapsed_time, 2)
    logging.info(f'Elapsed time writting VALID embeddings: {elapsed_time} seconds')
    
    if task_type == 'chief':
        _upload_blob_gcs(
            EMBEDDINGS_PATH, 
            f"{EMBEDDINGS_FILE_NAME}", 
            f"{EMBEDDINGS_FILE_NAME}",
            args.project
        )
    
    logging.info('All done - model saved') #all done
    
# ====================================================
# arg parser
# ====================================================
    
def parse_args():
    """Parses command line arguments."""

    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--experiment_name',
        type=str,
        required=False,
        default='unnamed-experiment',
        help='name of vertex ai experiment'
    )
    parser.add_argument(
        '--experiment_run',
        type=str,
        required=False,
        default='unnamed_run',
        help='name of vertex ai experiment run'
    )
    parser.add_argument(
        '--tb_name',
        type=str,
        required=True,
        help='projects/XXXXXX/locations/us-central1/tensorboards/XXXXXXXX'
    )
    parser.add_argument(
        '--distribute',
        type=str,
        required=False,
        default='single',
        help='training strategy'
    )
    parser.add_argument(
        '--train_output_bucket',
        type=str,
        required=True,
        # default='single',
        help='gcs bucket name'
    )
    parser.add_argument(
        '--workflow_dir',
        type=str,
        required=True,
        help='Path to saved workflow.pkl e.g., nvt-analyzed'
    )
    parser.add_argument(
        '--train_dir',
        type=str,
        required=True,
        help='Path to training data _file_list.txt'
    )
    parser.add_argument(
        '--valid_dir',
        type=str,
        required=True,
        help='Path to validation data _file_list.txt'
    )
    parser.add_argument(
        '--num_epochs',
        type=int,
        required=True,
        help='num_epochs'
    )
    parser.add_argument(
        '--per_gpu_batch_size',
        type=int,
        required=True,
        help='Per GPU Batch size'
    )
    parser.add_argument(
        '--layer_sizes',
        type=str,
        required=False,
        default='[512, 256, 128]',
        help='layer_sizes'
    )
    parser.add_argument(
        '--learning_rate',
        type=float,
        required=False,
        default=.001,
        help='learning_rate'
    )
    parser.add_argument(
        '--project',
        type=str,
        required=True,
        help='gcp project'
    )
    parser.add_argument(
        '--location',
        type=str,
        required=True,
        help='gcp location'
    )
    # parser.add_argument(
    #     '--gpus',
    #     type=str,
    #     required=False,
    #     default='[[0]]',
    #     help='GPU devices to use for Preprocessing'
    # )
    
    return parser.parse_args()

if __name__ == '__main__':
    logging.basicConfig(
        format='%(asctime)s - %(message)s',
        level=logging.INFO, 
        datefmt='%d-%m-%y %H:%M:%S',
        stream=sys.stdout
    )

    parsed_args = parse_args()

    # parsed_args.gpus = json.loads(parsed_args.gpus)

    # parsed_args.slot_size_array = [
    #     int(i) for i in parsed_args.slot_size_array.split(sep=' ')
    # ]

    logging.info('Args: %s', parsed_args)
    start_time = time.time()
    logging.info('Starting training')

    main(parsed_args)

    end_time = time.time()
    elapsed_time = end_time - start_time
    logging.info('Training completed. Elapsed time: %s', elapsed_time )

Writing src/trainer/train_task.py


## Training Image

### versioned image

In [13]:
# Docker definitions for training
MERLIN_VERSION = '22_12_v4'
IMAGE_NAME = f'{FRAMEWORK}-{MODEL_NAME}-training-{VERSION}-{MERLIN_VERSION}'
IMAGE_URI = f'gcr.io/{PROJECT_ID}/{IMAGE_NAME}'

DOCKERNAME = f'merlintf-{MERLIN_VERSION}'
MACHINE_TYPE ='e2-highcpu-32'
FILE_LOCATION = './src'

* nvtabular==1.5.0
* nvtabular==1.3.3
* cloudml-hypertune

```
RUN pip install google-cloud-bigquery gcsfs
RUN pip install google-cloud-aiplatform[cloud_profiler] kfp
```

In [14]:
# %%writefile {REPO_DOCKER_PATH_PREFIX}/Dockerfile.{DOCKERNAME}

# FROM nvcr.io/nvidia/merlin/merlin-tensorflow:22.09

# WORKDIR /src

# RUN pip install -U pip
# RUN pip install git+https://github.com/NVIDIA-Merlin/models.git@efe4bc91cc7e161f6e1c6dab3ff2a8ef04fd84b5 gcsfs google-cloud-aiplatform[cloud_profiler] kfp
# RUN echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] http://packages.cloud.google.com/apt cloud-sdk main" | tee -a /etc/apt/sources.list.d/google-cloud-sdk.list && curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key --keyring /usr/share/keyrings/cloud.google.gpg  add - && apt-get update -y && apt-get install google-cloud-sdk -y

# COPY trainer/* ./

# ENV LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/hugectr/lib:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/lib:/repos/dist/lib:/opt/tritonserver/lib

In [15]:
# RUN pip install git+https://github.com/NVIDIA-Merlin/models.git@efe4bc91cc7e161f6e1c6dab3ff2a8ef04fd84b5 gcsfs google-cloud-aiplatform fastapi

In [16]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/Dockerfile.{DOCKERNAME}

FROM nvcr.io/nvidia/merlin/merlin-tensorflow:22.12

WORKDIR /

# RUN pip install -U pip
RUN pip install merlin-models gcsfs google-cloud-aiplatform fastapi


COPY trainer /trainer

RUN apt update && apt -y install nvtop

Writing src/Dockerfile.merlintf-22_12_v4


In [17]:
# %%writefile {REPO_DOCKER_PATH_PREFIX}/{TRAIN_SUB_DIR}/requirements.txt
# fastapi
# merlin-models
# gcsfs
# google-cloud-aiplatform

# Build Train Image

### `cloudbuild.yaml`

In [18]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/cloudbuild.yaml

steps:
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', '$_IMAGE_URI', '$_FILE_LOCATION', '-f', '$_FILE_LOCATION/Dockerfile.$_DOCKERNAME']
images:
- '$_IMAGE_URI'

Overwriting src/cloudbuild.yaml


In [19]:
# os.chdir('/home/jupyter/jt-merlin/merlin-on-vertex')
os.getcwd()

'/home/jupyter/merlin-on-vertex-ORIGINAL/merlin-on-vertex'

In [29]:
! gcloud builds submit --config src/cloudbuild.yaml \
    --substitutions _DOCKERNAME=$DOCKERNAME,_IMAGE_URI=$IMAGE_URI,_FILE_LOCATION=$FILE_LOCATION \
    --timeout=2h \
    --machine-type=$MACHINE_TYPE

# Vertex Train Job

### Prepare `worker_pool_specs`

In [21]:
def prepare_worker_pool_specs(
    image_uri,
    # args,
    cmd,
    replica_count=1,
    machine_type="n1-standard-16",
    accelerator_count=1,
    accelerator_type="ACCELERATOR_TYPE_UNSPECIFIED",
    reduction_server_count=0,
    reduction_server_machine_type="n1-highcpu-16",
    reduction_server_image_uri="us-docker.pkg.dev/vertex-ai-restricted/training/reductionserver:latest",
):

    if accelerator_count > 0:
        machine_spec = {
            "machine_type": machine_type,
            "accelerator_type": accelerator_type,
            "accelerator_count": accelerator_count,
        }
    else:
        machine_spec = {"machine_type": machine_type}

    container_spec = {
        "image_uri": image_uri,
        # "args": args,
        "command": cmd,
    }

    chief_spec = {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "container_spec": container_spec,
    }

    worker_pool_specs = [chief_spec]
    if replica_count > 1:
        workers_spec = {
            "replica_count": replica_count - 1,
            "machine_spec": machine_spec,
            "container_spec": container_spec,
        }
        worker_pool_specs.append(workers_spec)
    if reduction_server_count > 1:
        workers_spec = {
            "replica_count": reduction_server_count,
            "machine_spec": {
                "machine_type": reduction_server_machine_type,
            },
            "container_spec": {"image_uri": reduction_server_image_uri},
        }
        worker_pool_specs.append(workers_spec)

    return worker_pool_specs

### Acclerators and Device Strategy

In [22]:
import time

# ====================================================
# Single | Single machine, single GPU
# ====================================================
WORKER_MACHINE_TYPE = 'a2-highgpu-1g'
REPLICA_COUNT = 1
ACCELERATOR_TYPE = 'NVIDIA_TESLA_A100'
PER_MACHINE_ACCELERATOR_COUNT = 1
REDUCTION_SERVER_COUNT = 0                                                      
REDUCTION_SERVER_MACHINE_TYPE = "n1-highcpu-16"
DISTRIBUTE_STRATEGY = 'single'

## Train Args

### Previously defined Vars

In [30]:
print(f"PROJECT: {PROJECT_ID}")
print(f"VERSION: {VERSION}")
print(f"IMAGE_URI: {IMAGE_URI}")
print(f"MODEL_NAME: {MODEL_NAME}")
print(f"FRAMEWORK: {FRAMEWORK}")
print(f"MODEL_DISPLAY_NAME: {MODEL_DISPLAY_NAME}")
print(f"WORKSPACE: {WORKSPACE}")
print(f"IMAGE_URI: {IMAGE_URI}")

PROJECT: hybrid-vertex
VERSION: jtv16
IMAGE_URI: gcr.io/hybrid-vertex/merlin-tf-2tower-training-jtv16-22_12_v4
MODEL_NAME: 2tower
FRAMEWORK: merlin-tf
MODEL_DISPLAY_NAME: vertex-merlin-tf-2tower-jtv16
WORKSPACE: gs://jt-merlin-scaling/vertex-merlin-tf-2tower-jtv16
IMAGE_URI: gcr.io/hybrid-vertex/merlin-tf-2tower-training-jtv16-22_12_v4


In [31]:
TIMESTAMP = time.strftime("%Y%m%d-%H%M%S")
EXPERIMENT_PREFIX = 'latest'
EXPERIMENT_NAME = f'{EXPERIMENT_PREFIX}-{MODEL_NAME}-{FRAMEWORK}-{VERSION}'
RUN_NAME_PREFIX = f'run-{TIMESTAMP}' # timestamp assigned during job

# data and schema from nvtabular pipes
DATA_DIR = 'gs://jt-merlin-scaling/nvt-last5-latest-12/nvt-processed'
# DATA_DIR = 'gs://spotify-beam-v3/merlin-processed' #/train

TRAIN_DATA = f'{DATA_DIR}/train/' #/_gcs_file_list.txt'
VALID_DATA = f'{DATA_DIR}/valid/' #/_gcs_file_list.txt'

WORKFLOW_DIR = 'gs://jt-merlin-scaling/nvt-last5-latest-12/nvt-analyzed'
# WORKFLOW_DIR = 'gs://spotify-beam-v3/merlin-processed/workflow/2t-spotify-workflow'

print(f"EXPERIMENT_NAME: {EXPERIMENT_NAME}")
print(f"RUN_NAME_PREFIX: {RUN_NAME_PREFIX}")
print(f"TRAIN_DATA: {TRAIN_DATA}")
print(f"VALID_DATA: {VALID_DATA}")
print(f"WORKFLOW_DIR: {WORKFLOW_DIR}")

EXPERIMENT_NAME: latest-2tower-merlin-tf-jtv16
RUN_NAME_PREFIX: run-20230223-222603
TRAIN_DATA: gs://jt-merlin-scaling/nvt-last5-latest-12/nvt-processed/train/
VALID_DATA: gs://jt-merlin-scaling/nvt-last5-latest-12/nvt-processed/valid/
WORKFLOW_DIR: gs://jt-merlin-scaling/nvt-last5-latest-12/nvt-analyzed


### Managed TB

In [32]:
# ====================================================
# Managed Tensorboard
# ====================================================

# use existing TB instance
# TB_RESOURCE_NAME = 'projects/934903580331/locations/us-central1/tensorboards/6924469145035603968'

# # create new TB instance
TENSORBOARD_DISPLAY_NAME=f"{EXPERIMENT_NAME}-v1"
tensorboard = vertex_ai.Tensorboard.create(display_name=TENSORBOARD_DISPLAY_NAME, project=PROJECT_ID, location=LOCATION)
TB_RESOURCE_NAME = tensorboard.resource_name


print(f"TB_RESOURCE_NAME: {TB_RESOURCE_NAME}")
print(f"TB display name: {tensorboard.display_name}")

TB_RESOURCE_NAME: projects/934903580331/locations/us-central1/tensorboards/2559566312439283712
TB display name: latest-2tower-merlin-tf-jtv16-v1


### Worker args

In [33]:
OUTPUT_BUCKET = 'jt-merlin-scaling'
NUM_EPOCHS = 1
BATCH_SIZE = 4096*4      # TODO: `batch_size * 4 ? jw
LEARNING_RATE = 0.001
LAYERS = "[512, 256, 128]"

# python trainer/train_task.py    # python: can't open file 'trainer/train_task.py'
# python -m train_task            # /usr/bin/python: No module named train_task
# python -m trainer.train_task    # /etc/bash.bashrc: line 9: PS1: unbound variable
    
WORKER_CMD = [
    'sh',
    '-euc',
    f'''pip freeze && python -m trainer.train_task --tb_name={TB_RESOURCE_NAME} --per_gpu_batch_size={BATCH_SIZE} \
    --train_output_bucket={OUTPUT_BUCKET} --train_dir={TRAIN_DATA} --valid_dir={VALID_DATA} --workflow_dir={WORKFLOW_DIR} \
    --num_epochs={NUM_EPOCHS} --learning_rate={LEARNING_RATE} --distribute={DISTRIBUTE_STRATEGY} \
    --experiment_name={EXPERIMENT_NAME} --experiment_run={RUN_NAME_PREFIX} --project={PROJECT_ID} --location={LOCATION}'''
]
    # --layer_sizes={LAYERS} \

# ====================================================
# Worker pool specs
# ====================================================
    
WORKER_POOL_SPECS = prepare_worker_pool_specs(
    image_uri=IMAGE_URI,
    # args=WORKER_ARGS,
    cmd=WORKER_CMD,
    replica_count=REPLICA_COUNT,
    machine_type=WORKER_MACHINE_TYPE,
    accelerator_count=PER_MACHINE_ACCELERATOR_COUNT,
    accelerator_type=ACCELERATOR_TYPE,
    reduction_server_count=REDUCTION_SERVER_COUNT,
    reduction_server_machine_type=REDUCTION_SERVER_MACHINE_TYPE,
)

from pprint import pprint
pprint(WORKER_POOL_SPECS)
# jt-merlin-scaling/nvt-last5-latest-12

[{'container_spec': {'command': ['sh',
                                 '-euc',
                                 'pip freeze && python -m trainer.train_task '
                                 '--tb_name=projects/934903580331/locations/us-central1/tensorboards/2559566312439283712 '
                                 '--per_gpu_batch_size=16384     '
                                 '--train_output_bucket=jt-merlin-scaling '
                                 '--train_dir=gs://jt-merlin-scaling/nvt-last5-latest-12/nvt-processed/train/ '
                                 '--valid_dir=gs://jt-merlin-scaling/nvt-last5-latest-12/nvt-processed/valid/ '
                                 '--workflow_dir=gs://jt-merlin-scaling/nvt-last5-latest-12/nvt-analyzed     '
                                 '--num_epochs=1 --learning_rate=0.001 '
                                 '--distribute=single     '
                                 '--experiment_name=latest-2tower-merlin-tf-jtv16 '
                       

## Submit train job

In [34]:
BASE_OUTPUT_DIR = f'gs://{OUTPUT_BUCKET}/{EXPERIMENT_NAME}'

# initialize vertex sdk
vertex_ai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=f'{BASE_OUTPUT_DIR}/staging',
    # experiment=EXPERIMENT_NAME,
)

job_prefix = '2212-tb'
JOB_NAME = f'{job_prefix}-train-{MODEL_DISPLAY_NAME}'

# labels for train job
gpu_type = ACCELERATOR_TYPE.lower()
gpu_per_replica = PER_MACHINE_ACCELERATOR_COUNT
replica_cnt = REPLICA_COUNT

print(f'BASE_OUTPUT_DIR : {BASE_OUTPUT_DIR}')
print(f'JOB_NAME : {JOB_NAME}\n')
print(f'gpu_type : {gpu_type}')
print(f'gpu_per_replica : {gpu_per_replica}')
print(f'replica_cnt : {replica_cnt}')

BASE_OUTPUT_DIR : gs://jt-merlin-scaling/latest-2tower-merlin-tf-jtv16
JOB_NAME : 2212-tb-train-vertex-merlin-tf-2tower-jtv16

gpu_type : nvidia_tesla_a100
gpu_per_replica : 1
replica_cnt : 1


In [35]:
job = vertex_ai.CustomJob(
    display_name=JOB_NAME,
    worker_pool_specs=WORKER_POOL_SPECS,
    base_output_dir=BASE_OUTPUT_DIR,
    staging_bucket=f'{BASE_OUTPUT_DIR}/staging',
    labels={
        # 'mm_image' : 'nightly',
        'gpu' : f'{gpu_type}',
        'gpu_per_replica' : f'{gpu_per_replica}',
        'replica_cnt' : f'{replica_cnt}',
    }
)

job.run(
    tensorboard=TB_RESOURCE_NAME,
    service_account=VERTEX_SA,
    restart_job_on_worker_restart=False,
    enable_web_access=True,
    sync=False,
)